In [1]:
import numpy as np 
import pandas as pd


In [2]:
#reading the file
df = pd.read_csv('../TextFiles/smsspamcollection.tsv',sep='\t')

In [3]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [4]:
# take the raw text information and vectorize it
# before that i check for missing values
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [5]:
#check Label Columns
#we 747 instances of spam and 4825 instances of ham
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [7]:
#split the data into training set and test set
#I use the raw text as the feature it self
from sklearn.model_selection import train_test_split

In [11]:
X = df['message']

In [12]:
y = df['label']

In [13]:
#perform the train test split
#0.33 = 30 % or 33% data 
#randomizing 42 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=42)

In [14]:
#perform count vectorization, build a dictionary of features and transforms documents to feature vectors
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
#create an instance of count vectorizer
count_vect = CountVectorizer()


In [16]:
#it is still in raw text form. but i want to pass it into CountVectorizer
#and then transform this raw text
x

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
5       FreeMsg Hey there darling it's been 3 week's n...
6       Even my brother is not like to speak with me. ...
7       As per your request 'Melle Melle (Oru Minnamin...
8       WINNER!! As a valued network customer you have...
9       Had your mobile 11 months or more? U R entitle...
10      I'm gonna be home soon and i don't want to tal...
11      SIX chances to win CASH! From 100 to 20,000 po...
12      URGENT! You have won a 1 week FREE membership ...
13      I've been searching for the right words to tha...
14                    I HAVE A DATE ON SUNDAY WITH WILL!!
15      XXXMobileMovieClub: To use your credit, click ...
16                             Oh k...i'm watching here:)
17      Eh u r

In [17]:
# Fit Vectorizer t0 the Data (build a vocab, count the number of words...)
#count_vect.fit(X_train)
#X_train_counts = count_vect.transform(X_train)
# Transform the Original text MEssage ->>>> Vector
#you can do it also both in onestep line like thisinstead of 2 different lines above
X_train_counts = count_vect.fit_transform(X_train)

In [18]:
X_train_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [19]:
#show messages
X_train.shape

(3733,)

In [20]:
#vocab messages 3733 and vocab counts 7082
X_train_counts.shape

(3733, 7082)

In [21]:
#transform the counts ( 0,0,0,1,0,1,0,1) to frequencies TFID 
from sklearn.feature_extraction.text import TfidfTransformer

In [23]:
tfidf_transformer = TfidfTransformer()

In [24]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [25]:
#check the shape of the object
X_train_tfidf.shape


(3733, 7082)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
vectorizer = TfidfVectorizer()

In [30]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [31]:
#train a clasifier
from sklearn.svm import LinearSVC

In [32]:
#create an instance 
clf = LinearSVC()

In [33]:
clf.fit(X_train_tfidf,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [34]:
#combine everything into one single pipeline step
from sklearn.pipeline import Pipeline

In [35]:
#combine everything into one single pipeline step and tell the pipeline what steps to take
text_clf = Pipeline([('tfdf',TfidfVectorizer()),('clf',LinearSVC())])

In [36]:
text_clf.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('tfdf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                

In [39]:
predictions = text_clf.predict(X_test)

In [40]:
from sklearn.metrics import confusion_matrix,classification_report

In [41]:
print(confusion_matrix(y_test,predictions))

[[1586    7]
 [  12  234]]


In [42]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [43]:
#let's check out accuracy
from sklearn import metrics

In [44]:
metrics.accuracy_score(y_test,predictions)

0.989668297988037

In [45]:
#now we can predict spam or ham in this HI how are
text_clf.predict(["Hi how are you doing today?"])

array(['ham'], dtype=object)